# 04_Parameter_Analysis

Run systematic parameter sweeps (transaction cost, risk penalty) and summarize the effects on agent robustness.


In [ ]:
from src.environment.multi_asset_env import MultiAsset21DeepHedgingEnv
from src.agents.ppo_agent import PPOAgent
from src.agents.evaluation import evaluate_model, summary_to_dataframe
from src.utils.data_utils import download_market_data
from src.environment.option_pricing import create_synthetic_option_chain
from src.config.settings import get_config
import matplotlib.pyplot as plt

df = download_market_data(**get_config('data'))
option_chain = create_synthetic_option_chain(df, get_config('option'))

strikes = get_config('option')['strike_offsets']
expiries = get_config('option')['expiry_days']
types_ = get_config('option')['option_types']
asset_universe = [{'strike_offset': s,'expiry_days':e,'type':t} for e in expiries for s in strikes for t in types_]


In [ ]:
costs = [0.0001, 0.001, 0.005]
risks = [0.005, 0.01, 0.05]
results = []

for cost in costs:
    for risk in risks:
        print(f\"Training for cost={cost}, risk_penalty={risk}\")
        env = MultiAsset21DeepHedgingEnv(df, option_chain, asset_universe,
                                         config_name='env')
        env.transaction_cost = cost
        env.risk_penalty = risk
        agent = PPOAgent(env)
        model = agent.create_model()
        model.learn(total_timesteps=15000)
        metrics = evaluate_model(model, env, episodes=10)
        results.append({'tx_cost': cost, 'risk_pen': risk,
                        'final_pnl': metrics['mean_reward']*10,
                        'avg_reward': metrics['mean_reward'],
                        'sharpe': metrics['sharpe_ratio']})
        print(f\"Final PnL: {metrics['mean_reward']*10:.2f}, Sharpe: {metrics['sharpe_ratio']:.2f}\")

In [ ]:
summary = summary_to_dataframe(results)
print(summary)


## 4.2 Plot Parameter Sensitivity


In [ ]:
import seaborn as sns
pivot = summary.pivot(index='tx_cost', columns='risk_pen', values='sharpe')
sns.heatmap(pivot, annot=True, cmap='viridis')
plt.title(\"Sharpe Sensitivity to Params\")
plt.ylabel(\"Transaction Cost\")
plt.xlabel(\"Risk Penalty\")
plt.show()
